# Genetic Algorithms

## Evolutionary Algorithms vs. Genetic Algorithms

### Evolutionary Algorithms:
1. Computational Models of Natural Evolution Processes
2. Simulation of Species Evolution
3. Survival of the Fittest
4. Self Organization, Adaptive Behavior

### Genetic Algorithms:
1. Branch of Evolutionary Algorithms
2. Better and Better Solutions based on the Evolution of Previous Generations

<img src="genetic_algo.png">

### Individual:
1. Individuals represent the solutions
2. A set of individuals make up a population
3. The chromosome represents a solution: the chromosome is a set of zeros and ones, indicating elements that will be taken and those that will not.

### Fitness Functions:
1. Quality measurement to find out how the chromosome solves the problem
2. Whether it is an acceptable solution and can be used for evolution

### Crossover:
1. It combines pieces of the chromosomes of two parents, generating more fit children
2. The population tends to evolve

<img src="crossover.png">

### Mutation:
1. Mutation creates diversity by randomly changing genes of the chromosomes
2. It is applied less frequently than crossover
3. It changes the genes according to a probability

#### Let's look at a maximization problem:
1. We have a truck of specific capacity
2. We have items with a value and weight
3. We need to load the truck with as many products as possible with maximum profit margin

<img src="business_case.png">

#### Product Class

In [1]:
class Product():
    def __init__(self, name, space, price):
        self.name = name
        self.space = space
        self.price = price

In [2]:
p1 = Product('Refrigerator A', 0.751, 999.9)

In [3]:
p1.name, p1.space, p1.price

('Refrigerator A', 0.751, 999.9)

In [4]:
p2 = Product('Cell phone', 0.00000899, 2199.12)

In [5]:
p2.name, p2.space, p2.price

('Cell phone', 8.99e-06, 2199.12)

In [6]:
products_list = []
products_list.append(Product('Refrigerator A', 0.751, 999.9))
products_list.append(Product('Cell phone', 0.00000899, 2199.12))
products_list.append(Product('TV 55', 0.400, 4346.99))
products_list.append(Product("TV 50' ", 0.290, 3999.90))
products_list.append(Product("TV 42' ", 0.200, 2999.00))
products_list.append(Product("Notebook A", 0.00350, 2499.90))
products_list.append(Product("Ventilator", 0.496, 199.90))
products_list.append(Product("Microwave A", 0.0424, 308.66))
products_list.append(Product("Microwave B", 0.0544, 429.90))
products_list.append(Product("Microwave C", 0.0319, 299.29))
products_list.append(Product("Refrigerator B", 0.635, 849.00))
products_list.append(Product("Refrigerator C", 0.870, 1199.89))
products_list.append(Product("Notebook B", 0.498, 1999.90))
products_list.append(Product("Notebook C", 0.527, 3999.00))

In [7]:
for i, product in enumerate(products_list):
    print(i + 1, product.name, product.space, product.price)

1 Refrigerator A 0.751 999.9
2 Cell phone 8.99e-06 2199.12
3 TV 55 0.4 4346.99
4 TV 50'  0.29 3999.9
5 TV 42'  0.2 2999.0
6 Notebook A 0.0035 2499.9
7 Ventilator 0.496 199.9
8 Microwave A 0.0424 308.66
9 Microwave B 0.0544 429.9
10 Microwave C 0.0319 299.29
11 Refrigerator B 0.635 849.0
12 Refrigerator C 0.87 1199.89
13 Notebook B 0.498 1999.9
14 Notebook C 0.527 3999.0


#### Individual class

In [8]:
from random import random

In [9]:
random()

0.15977828187637622

In [10]:
class Individual():
    def __init__(self, spaces, prices, space_limit, generation=0):
        self.spaces = spaces
        self.prices = prices
        self.space_limit = space_limit
        self.score_evaluation = 0
        self.used_space = 0
        self.generation = generation
        self.chromosome = []

        for i in range(len(spaces)):
            if random() < 0.5:
                self.chromosome.append('0')
            else:
                self.chromosome.append('1')

    def fitness(self):
        score = 0
        sum_spaces = 0

        for i in range(len(self.chromosome)):
            if self.chromosome[i] == '1':
                score += self.prices[i]
                sum_spaces += self.spaces[i]

            if sum_spaces > self.space_limit:
                score = 1

            self.score_evaluation = score
            self.used_space = sum_spaces
            
    def crossover(self, other_individual):
        cutoff = round(random() * len(self.chromosome))
        #print(cutoff)
        
        child1 = other_individual.chromosome[0:cutoff] + self.chromosome[cutoff::]
        child2 = self.chromosome[0:cutoff] + other_individual.chromosome[cutoff::]
        #print(child1)
        #print(child2)
        
        children = [Individual(self.spaces, self.prices, self.space_limit, self.generation + 1),
                   Individual(self.spaces, self.prices, self.space_limit, self.generation + 1)]
        
        children[0].chromosome = child1
        children[1].chromosome = child2
        
        return children
    
    def mutation(self, rate):
        print('Before: ', self.chromosome)
        for i in range(len(self.chromosome)):            
            if random() < rate:
                if self.chromosome[i] == '1':
                    self.chromosome[i] == '0'
                else:
                    self.chromosome[i] = '1'
                    
        print('After:  ', self.chromosome)
        return self

### Genetic Algorithm Class

In [39]:
class GeneticAlgorithm():
    
    def __init__(self, population_size):
        self.population_size = population_size
        self.population = []
        self.generation = 0
        self.best_solution = None
        self.list_of_solutions = []
        
    def initialize_population(self, spaces, prices, space_limit):
        for i in range(self.population_size):
            self.population.append(Individual(spaces, prices, space_limit))
            
        self.best_solution = self.population[0]
        
    def order_population(self):
        self.population = sorted(self.population, key=lambda population:population.score_evaluation, reverse=True)
        
    def best_individual(self, individual):
        if individual.score_evaluation > self.best_solution.score_evaluation:
            self.best_solution = individual
            
    def sum_evaluations(self):
        _sum = 0
        for individual in self.population:
            _sum += individual.score_evaluation
        return _sum   
    
    
    def select_parent(self, sum_evaluation):
        parent = -1
        random_value = random() * sum_evaluation
        _sum = 0
        i = 0
        
        #print('*** random value: ', random_value)
        while i < len(self.population) and _sum < random_value:
            #print('i: ', i, ' - sum: ', _sum)
            _sum += self.population[i].score_evaluation
            parent += 1
            i += 1
            
        return parent
    
    def visualize_generation(self):
        best = self.population[0]
        print('\nGeneration: ', self.population[0].generation, '\nTotal price: ', best.score_evaluation,
             '\nSpace: ', best.used_space, '\nChromosome: ', best.chromosome)

In [12]:
prices = []
names = []
spaces = []

for product in products_list:
    prices.append(product.price)
    names.append(product.name)
    spaces.append(product.space)
limit = 3

In [13]:
print(prices)

[999.9, 2199.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.0]


In [14]:
print(names)

['Refrigerator A', 'Cell phone', 'TV 55', "TV 50' ", "TV 42' ", 'Notebook A', 'Ventilator', 'Microwave A', 'Microwave B', 'Microwave C', 'Refrigerator B', 'Refrigerator C', 'Notebook B', 'Notebook C']


In [15]:
print(spaces)

[0.751, 8.99e-06, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527]


In [16]:
individual1 = Individual(spaces, prices, limit)
# print('Spaces: ', individual1.spaces)
# print('Prices: ', individual1.prices)
# print('Chromosome: ', individual1.chromosome)

for i in range(len(products_list)):
    if individual1.chromosome[i] == '1':
        print('Name: ', products_list[i].name)
individual1.fitness()

print('\nScore: ', individual1.score_evaluation)
print('Used space: ', individual1.used_space)
print('Chromosome: ', individual1.chromosome)

Name:  TV 42' 
Name:  Ventilator
Name:  Microwave A
Name:  Microwave B
Name:  Microwave C
Name:  Refrigerator C
Name:  Notebook B

Score:  7436.540000000001
Used space:  2.1927000000000003
Chromosome:  ['0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0']


In [17]:
individual2 = Individual(spaces, prices, limit)
# print('Spaces: ', individual2.spaces)
# print('Prices: ', individual2.prices)
# print('Chromosome: ', individual2.chromosome)

for i in range(len(products_list)):
    if individual2.chromosome[i] == '1':
        print('Name: ', products_list[i].name)
individual2.fitness()

print('\nScore: ', individual2.score_evaluation)
print('Used space: ', individual2.used_space)
print('Chromosome: ', individual2.chromosome)

Name:  Refrigerator A
Name:  TV 42' 
Name:  Notebook A
Name:  Microwave B
Name:  Microwave C
Name:  Refrigerator B
Name:  Notebook C

Score:  12075.99
Used space:  2.2028
Chromosome:  ['1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1']


In [18]:
children = individual1.crossover(individual2)

In [19]:
children[0].fitness()
print(children[0].score_evaluation)
print(children[0].chromosome)

8436.44
['1', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0']


In [20]:
children[1].fitness()
print(children[1].score_evaluation)
print(children[1].chromosome)

11076.09
['0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1']


In [21]:
individual1.mutation(0.01)

Before:  ['0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0']
After:   ['0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0']


In [22]:
population_size = 20
ga = GeneticAlgorithm(population_size)
ga.initialize_population(spaces, prices, limit)

In [23]:
ga.population[3].chromosome

['0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1']

In [24]:
for individual in ga.population:
    individual.fitness()
ga.order_population()    
for i in range(ga.population_size):
    print('Individual: ', (i + 1), '\nSpaces: ', ga.population[i].spaces, '\nPrices: ', 
          ga.population[i].prices, '\nChromosome: ', ga.population[i].chromosome, '\nUsed Space: ', 
          ga.population[i].used_space,'\nScore: ', ga.population[i].score_evaluation, '\n')

Individual:  1 
Spaces:  [0.751, 8.99e-06, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527] 
Prices:  [999.9, 2199.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.0] 
Chromosome:  ['0', '1', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '0', '1'] 
Used Space:  1.90990899 
Score:  18323.809999999998 

Individual:  2 
Spaces:  [0.751, 8.99e-06, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527] 
Prices:  [999.9, 2199.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.0] 
Chromosome:  ['1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0'] 
Used Space:  1.77320899 
Score:  18082.660000000003 

Individual:  3 
Spaces:  [0.751, 8.99e-06, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527] 
Prices:  [999.9, 2199.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199

In [25]:
ga.best_solution.score_evaluation

16283.47

In [26]:
ga.population[0].score_evaluation

18323.809999999998

In [27]:
ga.best_individual(ga.population[0])

In [28]:
ga.best_solution.score_evaluation

18323.809999999998

In [29]:
_sum = ga.sum_evaluations()
print('Sum of Evaluations: ', _sum)

Sum of Evaluations:  216529.34000000003


In [30]:
parent_1 = ga.select_parent(_sum)
parent_1

3

In [37]:
new_population = []
mutation_probability = 0.01
for individual in range(0, ga.population_size, 2):
    parent1 = ga.select_parent(_sum)
    parent2 = ga.select_parent(_sum)
    print('\n', parent1, parent2)
    children = ga.population[parent1].crossover(ga.population[parent2])
    print('parent 1: ', ga.population[parent1].chromosome)
    print('parent 2: ', ga.population[parent2].chromosome)
    print(' child 1: ', children[0].chromosome)
    print(' child 2: ', children[1].chromosome)
    
    new_population.append(children[0].mutation(mutation_probability))
    new_population.append(children[1].mutation(mutation_probability))


 14 1
parent 1:  ['0', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0']
parent 2:  ['1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0']
 child 1:  ['1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0']
 child 2:  ['0', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0']
Before:  ['1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0']
After:   ['1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0']
Before:  ['0', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0']
After:   ['0', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0']

 1 4
parent 1:  ['1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0']
parent 2:  ['1', '0', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '0', '1']
 child 1:  ['1', '0', '0', '1', '1', '1', '0', '0', '0', '1', '0', '0', '0', '0']
 child 2:  ['1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1']
Before:  ['

In [41]:
ga.visualize_generation()

AttributeError: 'GeneticAlgorithm' object has no attribute 'visualize_generation'